In [1]:
import os, time, random, logging , datetime , cv2 , csv , subprocess
import numpy as np


import tensorflow as tf
print("tf",tf.version.VERSION)
from tensorflow import keras

from utils import globo , xdv , tfh5 , sinet


''' GPU CONFIGURATION '''

#os.environ["CUDA_VISIBLE_DEVICES"]="1,2"
#tfh5.set_tf_loglevel(logging.ERROR)
#tfh5.tf.debugging.set_log_device_placement(False) #Enabling device placement logging causes any Tensor allocations or operations to be printed.
#tfh5.set_memory_growth()
#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


CFG_SINET = {
    
    'sinet_version': 'fsd-sinet-vgg42-tlpf_aps-1',
    
    'graph_filename' : os.path.join(globo.FSDSINET_PATH,"fsd-sinet-vgg42-tlpf_aps-1.pb"),
    'metadata_file'  : os.path.join(globo.FSDSINET_PATH,"fsd-sinet-vgg42-tlpf_aps-1.json"),
    
    'audio_fs_input':22050,
    'batchSize' : 64,
    'lastPatchMode': 'repeat',
    'patchHopSize' : 50,
    
    
    'anom_labels' : ["Alarm","Boom","Crowd","Dog","Drill","Explosion","Fire","Gunshot and gunfire","Hammer","Screaming","Screech",\
                    "Shatter","Shout","Siren","Slam","Squeak","Yell"],
    'anom_labels_i' : [4,18,51,59,65,72,78,92,94,145,146,147,148,152,154,161,198],
    
    'anom_labels2' : ["Boom","Explosion","Fire","Gunshot and gunfire","Screaming",\
                    "Shout","Siren","Yell"],
    'anom_labels_i2' : [18,72,78,92,147,148,152,198],
    
    'full_or_max' : 'max', #chose output to (timesteps,labels_total) ot (1,labels_total)
    'labels_total' : 200
    
}
CFG_WAV= {
    
    "full_or_max" : CFG_SINET["full_or_max"],
    "sinet_aas_len" : CFG_SINET["labels_total"], 
    
    "shuffle" : False,
    
    "ativa" : 'relu',
    "optima" : 'sgd',
    "batch_type" : 0,   # =0 all batch have frame_max or video length // =1 last batch has frame_max frames // =2 last batch has no repetead frames
    "frame_max" : 8000,
    "ckpt_start" : f"{0:0>8}",  #used in train_model: if 00000000 start from scratch, else start from ckpt with CFG_WAV stated
    
    "epochs" : 1,
    "batch_size" : 1
    
}

tf 2.2.0


2023-05-04 01:11:37.044014: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-04 01:11:37.044042: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[   INFO   ] MusicExtractorSVM: no classifier models were configured by default
2023-05-04 01:11:38.251169: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 01:11:38.254827: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-05-04 01:11:39.668597

In [2]:
data = np.load(os.path.join(globo.AAS_PATH,f"{CFG_SINET['sinet_version']}--fl_train.npz"), allow_pickle=True)["data"]

In [16]:
def rescale_sigmoid(data):
    
    def sigmoid(x):return 1 / (1 + np.exp(-x))
    
    num_features = data.shape[1]
    scaled_data = np.zeros_like(data)

    for feature_idx in range(num_features):
        feature_data = data[:, feature_idx]

        # Center the data around the mean
        centered_data = feature_data - np.mean(feature_data)

        # Apply sigmoid function
        scaled_feature_data = sigmoid(centered_data)
        scaled_data[:, feature_idx] = scaled_feature_data

    return scaled_data

In [19]:
def min_max_scaling(data):
    num_features = data.shape[1]
    scaled_data = np.zeros_like(data)

    for feature_idx in range(num_features):
        feature_data = data[:, feature_idx]

        min_val = np.min(feature_data)
        max_val = np.max(feature_data)

        # Avoid division by zero
        feature_range = max_val - min_val
        if feature_range == 0:
            feature_range = 1
        
        scaled_feature_data = (feature_data - min_val) / feature_range
        scaled_data[:, feature_idx] = scaled_feature_data
        
        #print(f"{feature_idx} , max{max_val} , min{min_val} , fr{feature_range}")
    
    return scaled_data

In [21]:
k = 214

label = data[k]['label']

p_es_arr = data[k]['p_es_array']
p_es_arr_norm = min_max_scaling(p_es_arr)
p_es_arr_sigm = rescale_sigmoid(p_es_arr)


print(np.shape(p_es_arr),np.shape(p_es_arr_norm),label)
for i in range(len(CFG_SINET['anom_labels_i2'])):
    label_i = CFG_SINET['anom_labels_i2'][i]
    print("\t",label_i, CFG_SINET['anom_labels2'][i])
    #print('\n\n',p_es_arr[:, label_i])
    maxx = np.amax(p_es_arr[:, label_i])
    avgg = np.mean(p_es_arr[:, label_i])
    
    mmmaxx = np.amax(p_es_arr_norm[:, label_i])
    mmavgg = np.mean(p_es_arr_norm[:, label_i])
    
    
    print("\nORIGNAL\n",p_es_arr[:,label_i])
    print("\nMinMaxNORM\n",p_es_arr_norm[:,label_i])
    print("\nSIGMOID\n",p_es_arr_sigm[:,label_i])
       
    
    print("@ MAX", maxx,"@ mmMAX",mmmaxx,\
        "@ AVG", avgg,"@ mmAVG",mmavgg)


(22, 200) (22, 200) 1
	 18 Boom

ORIGNAL
 [0.74649584 0.7636646  0.7584472  0.7521365  0.7239072  0.7666274
 0.7515168  0.7576033  0.7505058  0.7228563  0.7562321  0.7847883
 0.7934009  0.69900393 0.73295236 0.76874614 0.75454414 0.7278354
 0.7664895  0.7268493  0.78801787 0.74824226]

MinMaxNORM
 [0.5031085  0.6849868  0.6297156  0.56286347 0.263814   0.71637315
 0.55629855 0.62077594 0.5455883  0.25268134 0.60625    0.90876216
 1.         0.         0.35963473 0.7388184  0.58836865 0.30542812
 0.71491265 0.2949818  0.9429747  0.52160925]

SIGMOID
 [0.49865964 0.5029518  0.5016474  0.5000698  0.49301293 0.50369245
 0.49991482 0.5014365  0.4996621  0.49275026 0.5010937  0.508232
 0.5103844  0.48678973 0.49527386 0.5042221  0.5006717  0.49399477
 0.503658   0.4937483  0.50903916 0.49909624]
@ MAX 0.7934009 @ mmMAX 1.0 @ AVG 0.7518574 @ mmAVG 0.55990666
	 72 Explosion

ORIGNAL
 [0.6387316  0.6160574  0.64335984 0.5519111  0.5607095  0.6857827
 0.6795409  0.63383293 0.6198667  0.5923649  

In [ ]:
len(p_es_arr[0])

In [ ]:
aaa = np.max(p_es_arr , axis = 0)

print(np.shape(aaa))


bbb = min_max_scaling(p_es_arr)
print(np.shape(bbb))

In [ ]:
'''
from tensorflow.keras.preprocessing.sequence import pad_sequences

p_es = [] #np.empty((len(data) ,None, CFG_SINET["labels_total"]), dtype=np.float32)

for j in range(len(data)):
    p_es.append(data[j]['p_es_array'])
    

# Assuming X is a list of input sequences with varying lengths
X_padded = pad_sequences(p_es, dtype='float32', padding='post')

# Now, you can use X_padded to train or evaluate your model
# '''

In [ ]:
#xdv.create_train_valdt_test_from_xdvtest_bg()
data=xdv.train_valdt_test_from_xdvtest_bg_from_npy(True)


In [ ]:
sinet = sinet.Sinet(CFG_SINET)

In [ ]:
len(data['train'][7][1])